# User Input

In [2]:
dir_ = 'data/'
fname = dir_ + 'dsDNA_-8TA+10NTD_700pM_1Dto20Adilut_600mW_conf1_reall_5.dat'

In [ ]:
sample_name = 'dsDNA TA-8_NTD+10'
description = '48-spot smFRET measurement of dsDNA.'
dye_names = 'ATTO550, ATTO647N'
buffer_name = 'TE50'

author = 'Antonino Ingargiola'
author_affiliation = 'UCLA'
creator = 'Antonino Ingargiola'
creator_affiliation = 'UCLA'

# Imports

In [3]:
import time
import numpy as np
import tables
from tqdm import tqdm_notebook, tnrange  # progress bar

import phconvert as phc
phc.__version__

'0.7.3+15.ged8ec9f'

In [4]:
import niconverter as nic

# Preprocess spot data

In [5]:
t_start = time.time()

In [6]:
h5file, meta = nic.ni96ch_process_spots(fname, close=True)
ts_unit = 1 / meta['clock_frequency']
fname, meta

('data/dsDNA_-8TA+10NTD_700pM_1Dto20Adilut_600mW_conf1_reall_5.dat',
 {'acquisition_duration': 46.045105437499998,
  'clock_frequency': 80000000.0,
  'format': 'x',
  'nchannels': 96,
  'num_timestamps': 75626617,
  'offset': 65,
  'words_per_photon': 1})

In [7]:
time.time() - t_start

39.522247076034546

In [8]:
h5file = tables.open_file(fname + '_raw_temp_spots.hdf5', mode='a')
dt, endianess, meta = nic._detectformat(fname)
ts_unit = 1 / meta['clock_frequency']
t2, aem2 = nic.get_photon_data_arr(h5file, spots=np.arange(48))
meta['acquisition_duration'] = nic._duration(t2, ts_unit)
fname, meta

('data/dsDNA_-8TA+10NTD_700pM_1Dto20Adilut_600mW_conf1_reall_5.dat',
 {'acquisition_duration': 46.045105437499998,
  'clock_frequency': 80000000.0,
  'format': 'x',
  'nchannels': 96,
  'num_timestamps': 75626617,
  'offset': 65,
  'words_per_photon': 1})

In [9]:
time.time() - t_start

40.350224018096924

In [10]:
print('Measurement duration: %.1f s' %  meta['acquisition_duration'])
print('Transfer rate: %.2f MB/s' % (meta['num_timestamps'] / meta['acquisition_duration'] * 4 / (1024**2)))

Measurement duration: 46.0 s
Transfer rate: 6.27 MB/s


In [11]:
neg_jumps = []
for tx in tqdm_notebook(t2):
    d = np.diff(tx[:])
    assert (d >= 0).all()
    neg_jumps.append(d[d<0])
    
missed_ov_jumps = [len(j) for j in neg_jumps]
for ch, jumps in enumerate(missed_ov_jumps):
    if jumps > 0:
        print('CH%d: %s' % (ch, jumps))

In [12]:
time.time() - t_start

44.00565195083618

# Create Photon-HDF5

In [14]:
metadata = dict(
    description = description,
    acquisition_duration = meta['acquisition_duration'],

    sample = dict(
        sample_name=sample_name,
        dye_names=dye_names,
        buffer_name=buffer_name,
        num_dyes = len(dye_names.split(','))),

    identity = dict(
        author=author,
        author_affiliation=author_affiliation,
        creator=creator,
        creator_affiliation=creator_affiliation),
    )
metadata

{'acquisition_duration': 46.045105437499998,
 'description': '48-spot smFRET measurement of dsDNA.',
 'identity': {'author': 'Antonino Ingargiola',
  'author_affiliation': 'UCLA',
  'creator': 'Antonino Ingargiola',
  'creator_affiliation': 'UCLA'},
 'sample': {'buffer_name': 'TE50',
  'dye_names': 'ATTO550, ATTO647N',
  'num_dyes': 2,
  'sample_name': 'dsDNA TA-8_NTD+10'}}

In [15]:
fname

'data/dsDNA_-8TA+10NTD_700pM_1Dto20Adilut_600mW_conf1_reall_5.dat'

In [16]:
h5file.filename

'data/dsDNA_-8TA+10NTD_700pM_1Dto20Adilut_600mW_conf1_reall_5.dat_raw_temp_spots.hdf5'

In [17]:
data = nic.create_ph5data_smFRET_48spots(fname, h5file, ts_unit, metadata)

In [18]:
[f for f in data.keys() if not f.startswith('photon')]

['_filename',
 'acquisition_duration',
 'description',
 'sample',
 'identity',
 'setup',
 'provenance']

In [19]:
%%timeit -n1 -r1
phc.hdf5.save_photon_hdf5(data, h5file=h5file, overwrite=True, close=True)

Saving: data/dsDNA_-8TA+10NTD_700pM_1Dto20Adilut_600mW_conf1_reall_5.dat_raw_temp_spots.hdf5
1 loop, best of 1: 1.25 s per loop


In [20]:
time.time() - t_start

45.34969210624695

In [21]:
stop 

NameError: name 'stop' is not defined

# More stuff

In [ ]:
if data['_data_file'].isopen:
    data['_data_file'].close()

In [ ]:
h5file = tables.open_file(h5file.filename)

In [ ]:
ts_list, aem = nic.get_photon_data_arr(h5file, range(48))

In [ ]:
neg_jumps = []
for tx in tqdm_notebook(ts_list):
    d = np.diff(tx[:])
    assert (d > 0).all()
    neg_jumps.append(d[d<0])
assert sum(len(j) for j in neg_jumps) == 0

In [ ]:
jump_stats = [(len(j), min(j), max(j), np.mean(j)) if len(j) > 0 else (0, 0, 0, 0) for j in neg_jumps]

for ch, jumps in enumerate(jump_stats):
    if jumps[0] > 0:
        print('CH%02d: %4d [%4d %4d] <%d>' % (ch, *jumps))